# **Stable Diffusion: Advanced Inference Techniques**


## **Intro**

Stable Diffusion is a [Latent Diffusion model](https://github.com/CompVis/latent-diffusion) developed by researchers from the Machine Vision and Learning group at LMU Munich, *a.k.a* CompVis.
Model checkpoints were publicly released at the end of August by a collaboration of Stability AI, CompVis, and Runway with support from EleutherAI and LAION. For more information, you can check out [the official blog post](https://stability.ai/blog/stable-diffusion-public-release).

Since its public release the community has done an incredible job at working together to make the stable diffusion checkpoints **faster**, **more memory efficient**, and **more performant**.

[🧨 Diffusers](https://github.com/huggingface/diffusers) offers a simple API to run stable diffusion with all memory, compute and quality improvements.

This notebook walks you through the improvements one-by-one and you can best leverage *Stable Diffusion* for **inference**.

### Setup

First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster. If the following command fails, use the `Runtime` menu above and select `Change runtime type`.

In [ ]:
!nvidia-smi

Next, you should install `diffusers` as well `scipy`, `ftfy` and `transformers`.

In [ ]:
!pip install diffusers[torch]==0.11.1 transformers scipy ftfy accelerate

## Prompt Engineering 🎨

When running *Stable Diffusion* in inference, we usually want to generate a certain type, style of image and then improve upon it. Improving upon a previously generated image means running inference over and over again with a different prompt and potentially a different seed until we are happy with our generation.

So to begin with, it is most important to speed up stable diffusion as much as possible to generate as many pictures as possible in a given amount of time.

This can be done by both improving the **computational efficiency** (speed) and the **memory efficiency** (GPU RAM).

Let's start by looking into the computational efficiency first.

Through-out the notebook, we will focus on [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5):

In [ ]:
model_id = "stabilityai/stable-diffusion-2"

Let's load the pipeline as defined on https://huggingface.co/runwayml/stable-diffusion-v1-5 .

## Speed Optimization

In [ ]:
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(model_id)

We aim at generating a beautiful photograph of an *old warrior chief* and will later try to find the best prompt to generate such a photograph. For now, let's keep the prompt simple:

In [ ]:
prompt = "portrait photo of a old warrior chief"

To begin with, we should make sure we run inference on GPU, so let's move the pipeline to GPU, just like you would with any PyTorch module.

In [ ]:
pipe = pipe.to("cuda")

To generate an image, you should use the `__call__` method. You can have a look at its docstring to better understand what arguments can be passed [here](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.__call__).

To make sure we can reproduce more or less the same image in every call, let's make use of the generator. See documentation on reproducibality [here]( ) for more information.

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(0)

Now, let's take a spin on it.

In [ ]:
image = pipe(prompt, generator=generator).images[0]
image

Cool, this now took roughly 30 seconds on a T4 GPU (you might see faster inference if you're allocated GPU is better than a T4).

The default run we did above used full float32 precision and run the default 50 inference steps. The easiest speed-ups come from switiching to float16 (or half) precision and simply running less inference steps. Let's load the model now instead in float16.

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")

And we can again call the pipeline to generate an image.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)

image = pipe(prompt, generator=generator).images[0]
image

Cool, this is almost three times as fast for arguably the exact same image quality.

We strongly suggest to always run your pipelines in float16 as so far we have very rarely seen degradations in quality because of it.

Next, let's see if we really need to use 50 inference steps or whether we could use significantly less.

Let's have a look at all schedulers, the stable diffusion pipeline is compatible with.

In [ ]:
pipe.scheduler.compatibles

Cool, that's a lot of schedulers.

🧨 Diffusers is constantely adding a bunch of novel schedulers/samplers that can be used with Stable Diffusion. For more information, we recommend to take a look at the official documentation [here](https://huggingface.co/docs/diffusers/main/en/api/schedulers/overview).

Alright, right now Stable Diffusion is using the `PNDMScheduler` which usually requires around 50 inference steps. However, other schedulers such as `DPMSolverMultistepScheduler` or `DPMSolverSinglestepScheduler` seem to get away with just 20 to 25 inference steps. Let's try them out.

You can set a new scheduler by making use of the [from_config](https://huggingface.co/docs/diffusers/main/en/api/configuration#diffusers.ConfigMixin.from_config) function.

In [ ]:
from diffusers import DPMSolverMultistepScheduler

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

Now, let's try to reduce the number of inference steps to just 20.

In [ ]:
generator = torch.Generator("cuda").manual_seed(0)

image = pipe(prompt, generator=generator, num_inference_steps=20).images[0]
image

The image now does look a little different, but it's arguably still of equally high quality. We now cut inference time to just a few seconds though 😍.

## Memory Optimization

More memory indirectly also means more speed since we're often trying to maximize how many images we can generate per second, so more images per inference, more images per second usually.

The easiest way to see how much we can increase memory is to simply try it out, and see when we get a *"Out-of-memory (OOM)"* error.

One can run batched inference by simply passing a list of prompts and generators. Let's define a quick function that generates a batch for us.

In [ ]:
def get_inputs(batch_size=1):
  generator = [torch.Generator("cuda").manual_seed(i) for i in range(batch_size)]
  prompts = batch_size * [prompt]
  num_inference_steps = 20

  return {"prompt": prompts, "generator": generator, "num_inference_steps": num_inference_steps}

We also need a method that allows us to easily display a batch of images.

In [ ]:
from PIL import Image

def image_grid(imgs, rows=2, cols=2):
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

Cool, let's see how much memory we can use starting with batch_size=4.

In [ ]:
images = pipe(**get_inputs(batch_size=4)).images
image_grid(images)

Going over a batch_size of 4 will error out in this colab. Also we can see we only generate slighly more images per second (3.75s/image) compared to 4s/image previously.

However, the community has found some nice tricks to improve the memory constraints further${}^1$. By far most of the memory is taken up by the cross attention layers. Instead of running this operation in batch, one can run it sequentially to save a significant amout of memory.

It can easily be enabled by calling `enable_attention_slicing` as is documented [here](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.enable_attention_slicing).

---
${}^1$ After stable diffusion was relesaed, the community found improvements within days and shared the freely over GitHub - open-source at its finest!
I believe theh original idea came from [this](https://github.com/basujindal/stable-diffusion/pull/117). GitHub thread.


In [ ]:
pipe.enable_attention_slicing()

Great now that attention slicing is enabled, let's try to double the batch size again, going for `batch_size=8`.

In [ ]:
images = pipe(**get_inputs(batch_size=8)).images
image_grid(images, rows=2, cols=4)

Nice, it works. However, the speed gain is again not very big (it might however be much more significant on other GPUs).

We're at roughly 3.5 seconds per image 🔥 which is probably the fastest we can be with a simple T4 without sacrificing quality.

Next, let's look into how to improve the quality!

## Quality Improvements

Now that our image generation pipeline is blazing fast, let's try to get maximum image quality.

First of all, image quality is extremely subjective, so it's difficult to make general claims here.

The most obvious step to take to improve quality is to use *better checkpoints*. Since the release of Stable Diffusion many improved versions have been released, which are summarized here:

- *Official Release - 22 Aug 2022*: [Stable-Diffusion 1.4](https://huggingface.co/CompVis/stable-diffusion-v1-4)
- *20 October 2022*: [Stable-Diffusion 1.5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
- *24 Nov 2022*: [Stable-Diffusion 2.0](https://huggingface.co/stabilityai/stable-diffusion-2-0)
- *7 Dec 2022*: [Stable-Diffusion 2.1](https://huggingface.co/stabilityai/stable-diffusion-2-1)

Newer versions don't necessarily mean better image quality with the same parameters. People mentioned that *2.0* is slighly worse than *1.5* for certain prompts, but given the right prompt engineering *2.0* and *2.1* seem to be better.

Overall, we strongly recommend to just try the models out and read up on advice online $^{2}$

Additionally, the community has started fine-tuning many of the above versions on certain styles with some of them having an extremely high quality and gaining a lot of traction.

We recommend to simply have a look at all [diffusers checkpoints sorted by downloads and try out the different checkpoints](https://huggingface.co/models?library=diffusers).

For the following, we will stick to v1.5 for simplicity.


---
$^{2}$ E.g. it has been shown that using negative prompts is very important for 2.0 and 2.1 to get the highest possible quality. See for example [this nice blog post](https://minimaxir.com/2022/11/stable-diffusion-negative-prompt/).


Next, we can also try to optimize single components of the pipeline, e.g. switching out the latent decoder. For more details on how the whole Stable Diffusion pipeline works, please have a look at [this blog post](https://huggingface.co/blog/stable_diffusion).

Let's load [stabilityai's newest auto-decoder](https://huggingface.co/stabilityai/stable-diffusion-2-1).

In [ ]:
from diffusers import AutoencoderKL

vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")

Now we can set it to the vae of the pipeline to use it.

In [ ]:
pipe.vae = vae

Let's run the same prompt as before to compare quality.

In [ ]:
images = pipe(**get_inputs(batch_size=8)).images
image_grid(images, rows=2, cols=4)

Seems like the difference is only very minor, but the new generations are arguably a bit *sharper*.

Cool, finally, let's look a bit into prompt engineering.

Our goal was to generate a photo of an old warrior chief. Let's now try to bring a bit more color into the photos and make the look more impressive.

Originally our prompt was "*portrait photo of a old warrior chief*".

To improve the prompt, it often helps to add cues that could have been used online to save high quality photos, as well as adding more details since.
Essentially, when doing prompt engineering, one has to think:
- How was the photo or similar photos of the one I want probably stored on the internet?
- What additional detail can I give that steers the models into the style that I want.

Cool, let's add more details.

In [ ]:
prompt += ", tribal panther make up, blue on red, side profile, looking away, serious eyes"

and let's also add some cues that usually help to generate higher quality images.

In [ ]:
prompt += " 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta"
prompt

Cool, let's now try this prompt.

In [ ]:
images = pipe(**get_inputs(batch_size=8)).images
image_grid(images, rows=2, cols=4)

Pretty impressive! We got definitely some very high quality image generations there. The 2nd image is my personal favorite, so I'll re-use this seed and see whether I can tweak the prompts slighly by using "oldest warrior", "old", "", and "young" instead of "old".

In [ ]:
prompts = [
    "portrait photo of the oldest warrior chief, tribal panther make up, blue on red, side profile, looking away, serious eyes 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta",
    "portrait photo of a old warrior chief, tribal panther make up, blue on red, side profile, looking away, serious eyes 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta",
    "portrait photo of a warrior chief, tribal panther make up, blue on red, side profile, looking away, serious eyes 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta",
    "portrait photo of a young warrior chief, tribal panther make up, blue on red, side profile, looking away, serious eyes 50mm portrait photography, hard rim lighting photography--beta --ar 2:3  --beta --upbeta",
]

generator = [torch.Generator("cuda").manual_seed(1) for _ in range(len(prompts))]  # 1 because we want the 2nd image

images = pipe(prompt=prompts, generator=generator, num_inference_steps=25).images
image_grid(images)

The first picture looks nice! The eye movement slighly changed and looks nice. This finished up our 101-guide on how to use Stable Diffusion 🤗.

For more information on optimization or other guides, I recommend to take a look at the following:

- [FlashAttention](https://huggingface.co/docs/diffusers/optimization/xformers): XFormers flash attention can optimize your model even further with more speed and memory improvements.
- [Dreambooth](https://huggingface.co/docs/diffusers/training/dreambooth) - Quickly customize for model by fine-tuning it.
- [General info on Stable Diffusion](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/overview) - Info on other tasks that are powered by Stable Diffusion.